In [ ]:
! pip install "whisper-at@git+https://github.com/Zivkr/whisper-at.git#subdirectory=package/whisper-at" torch torchvision 

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from tqdm import tqdm
import whisper_at as whisper
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import numpy as np
import audio_folder

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cpu


In [3]:
audio_tagging_time_resolution = 10
model = whisper.load_model("tiny", device=device)

# Freeze all layers except the last one
for param in model.parameters():
    param.requires_grad = False
    
model.at_model.mlp_layer = nn.Identity()

# Dataset

In [10]:
last_layer_size = model.at_model.layer_tr.mlp_ln.weight.shape[0]

def get_features(dataset):
    all_features = []
    all_labels = []
    for sound_batch, sample_rate, label, filename in tqdm(dataset):
        for sound in sound_batch:
            sound_features = model.transcribe(sound, at_time_res=audio_tagging_time_resolution, mlp_ln_output=last_layer_size, fp16=False)["audio_tag"].cpu().detach()
            all_features.append(sound_features)
            all_labels.append(label)

    return torch.cat(all_features).numpy(), np.asarray(all_labels,dtype=np.float32)

dataset = audio_folder.AudioFolder('/kaggle/input/zipped-datasets/audio')

In [ ]:
train_dataset, val_dataset = random_split(dataset, [0.8, 0.2])

train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=1, shuffle=False)

len(train_dataloader)

In [11]:
train_features, train_labels = get_features(train_dataloader)
train_features.shape, train_labels.shape

  0%|          | 8/2000 [00:32<2:14:19,  4.05s/it]


KeyboardInterrupt: 

In [ ]:
val_features, val_labels = get_features(val_dataloader)
val_features.shape, val_labels.shape

In [15]:
classifier = LogisticRegression(random_state=0, C=0.316, max_iter=1000, verbose=1)

classifier.fit(train_features, train_labels)
# Evaluate using the logistic regression classifier
predictions = classifier.predict( val_features)

accuracy = accuracy_score(val_labels, predictions)
print("Accuracy:", accuracy*100)

ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: 32